In [9]:
from lstm_creator import createLstmModelFromDatasets, scaleData, preprocessData
from predict_next_value import predict_next, load_x_from_file

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# use the last 90 minutes to predict next value
lookback_length = 20

# data is stores from every 15 seconds, but we only want every 4th item
sample_every_x = 3

paths = ["../../../data/dump.csv"]
model = createLstmModelFromDatasets(paths, lookback_length, sample_every_x, epochs=120)

# Uncomment this line to save the model on completion
# model.save('../../../models/dump_model_3_ll20_xth3')

Epoch 1/120
167/167 [==============================] - 5s 11ms/step - loss: 0.0382
Epoch 2/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0048
Epoch 3/120
167/167 [==============================] - 2s 10ms/step - loss: 0.0045
Epoch 4/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0044
Epoch 5/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0041
Epoch 6/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0036
Epoch 7/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0031
Epoch 8/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0028
Epoch 9/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0023
Epoch 10/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0018
Epoch 11/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0017
Epoch 12/120
167/167 [==============================] - 2s 11ms/step - loss: 0.0016
E

# Load model

In [11]:
from tensorflow.keras import models
from lstm_creator import createLstmModelFromDatasets, scaleData, preprocessData
import numpy as np

# Uncomment this line to load the model from a file for testing
# model = models.load_model('../../../models/dump_model_1')

# Testing

In [38]:
def evaluate_model(real_data, predicted_data, slope_to_buy_at, inherent_loss=2):
    real_data = real_data.reshape(len(real_data))
    predicted_data = predicted_data.reshape(len(predicted_data))

    real_diff = np.diff(real_data)
    predicted_diff = np.diff(predicted_data)

    profit = 0
    profits = 0
    losses = 0
    bought = False
    errors = []
    for i in range(len(real_data)-1):
            #  Error calculation
            if real_diff[i] != 0:
                errors.append( abs(real_diff[i] - predicted_diff[i]) / abs(real_diff[i]) )

            if predicted_diff[i] > slope_to_buy_at:
                if bought == False:
                    bought = real_data[i]
                else:
                    # Already bought, continue
                    continue
            if bought != False and predicted_diff[i] < slope_to_buy_at:
                # Now we sell
                profit += (real_data[i+1] - real_data[i]) - inherent_loss
                if ((real_data[i+1] - real_data[i]) - inherent_loss) > 0:
                    profits += 1
                else:
                    losses += 1
                bought = False

    return (errors, profit, profits, losses)

def eval_model_on_dataset(actual, prediction, slope_to_buy_at, subsampling, inherent_loss):
    # Determine the number of cases in which we predicted a correct increase
    errors, profit, profits, losses = evaluate_model(actual, prediction, slope_to_buy_at, inherent_loss)

    # print(f"Found {correct_slopes} out of {len(actual)-1}")
    percent_error = (sum(errors)/len(errors))*100
    # print(f"{precent_success}%")
    # print("Profit:", profit)
    return percent_error, profit, profits, losses

In [39]:
def testModel(model, length, slope_to_buy_at, path_to_testing_dataset, quiet=False):

    datasets, scaler = scaleData([path_to_testing_dataset], 3)

    hist, actual = preprocessData(datasets[0], length)

    pred = model.predict(hist)

    pred_transformed = scaler.inverse_transform(pred)
    actual_transformed = scaler.inverse_transform(actual)

    if not quiet:
        plt.figure(figsize=(12,8))
        plt.plot(actual_transformed, color='blue', label='Real')
        plt.plot(pred_transformed, color='red', label='Prediction')
        plt.title('ETH Price Prediction')
        plt.legend()
        plt.show()

    return eval_model_on_dataset(actual=actual_transformed, prediction=pred_transformed, slope_to_buy_at=slope_to_buy_at, subsampling=1, inherent_loss=1.5)

In [40]:
overall = (0, 0, 0, 0)
lookback_length = model.input_shape[1]
slope_to_buy_at = -1


result1 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test6.csv", quiet=True)
accuracy, profit, profits, losses = result1
print(f"Accuracy: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

result2 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test8.csv", quiet=True)
accuracy, profit, profits, losses = result2
print(f"Accuracy: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

result3 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test10.csv", quiet=True)
accuracy, profit, profits, losses = result3
print(f"Accuracy: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

overall = [sum(x) for x in zip(result1,overall)]
overall = [sum(x) for x in zip(result2,overall)]
overall = [sum(x) for x in zip(result3,overall)]
print("-----------------------------------")
print(f"Accuracy: {overall[0]/3:.2f}% Profit: ${overall[1]:.2f} (Profits/Losses: {overall[2]}/{overall[3]})")



Accuracy: 287.07% Profit: $-19.73 (Profits/Losses: 6/13)
Accuracy: 437.58% Profit: $-40.88 (Profits/Losses: 10/23)
Accuracy: 270.36% Profit: $-67.04 (Profits/Losses: 10/28)
-----------------------------------
Accuracy: 331.67% Profit: $-127.64 (Profits/Losses: 26/64)
